#### https://gallery.azure.ai/Experiment/Predictive-Maintenance-Step-1-of-3-data-preparation-and-feature-engineering-2

#### https://www.kaggle.com/behrad3d/nasa-cmaps

In [1]:
import tensorflow as tf
print(tf.__version__)


2.7.0


In [2]:
import tensorflow.keras as keras
print(keras.__version__)

2.7.0


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [4]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_c669b8b906494ca29667270d8d7cc9ba = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_c669b8b906494ca29667270d8d7cc9ba = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_c669b8b906494ca29667270d8d7cc9ba = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='W2LzmiMh0dm_lCUKnzVZuN9Zq8-e8jn9N0T30n50T8Cd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_c669b8b906494ca29667270d8d7cc9ba)

streaming_body_1 = client_c669b8b906494ca29667270d8d7cc9ba.get_object(Bucket='predictivemaintenanceaircraftengi-donotdelete-pr-05syrlmlnkok0c', Key='PM_train.txt')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
dataset_train=pd.read_csv(streaming_body_1,sep=' ',header=None).drop([26,27],axis=1)
col_names = ['id','cycle','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
dataset_train.columns=col_names
print('Shape of Train dataset: ',dataset_train.shape)
dataset_train.head()

EndpointConnectionError: Could not connect to the endpoint URL: "https://s3.private.us.cloud-object-storage.appdomain.cloud/predictivemaintenanceaircraftengi-donotdelete-pr-05syrlmlnkok0c/PM_train.txt"

In [ ]:
dataset_train['id'].value_counts()

In [ ]:

streaming_body_2 = client_c669b8b906494ca29667270d8d7cc9ba.get_object(Bucket='predictivemaintenanceaircraftengi-donotdelete-pr-05syrlmlnkok0c', Key='PM_test.txt')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
dataset_test=pd.read_csv(streaming_body_2,sep=' ',header=None).drop([26,27],axis=1)
dataset_test.columns=col_names
#dataset_test.head()
print('Shape of Test dataset: ',dataset_train.shape)
dataset_train.head()

In [ ]:

streaming_body_4 = client_c669b8b906494ca29667270d8d7cc9ba.get_object(Bucket='predictivemaintenanceaircraftengi-donotdelete-pr-05syrlmlnkok0c', Key='PM_truth.txt')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
pm_truth=pd.read_csv(streaming_body_4,sep=' ',header=None).drop([1],axis=1)
pm_truth.columns=['more']
pm_truth['id']=pm_truth.index+1
pm_truth.head()

In [ ]:
pm_truth.shape

In [ ]:
rul = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
rul.head()

In [ ]:
rul.shape

In [ ]:
pm_truth['rtf']=pm_truth['more']+rul['max']
pm_truth.head()

In [ ]:
pm_truth.shape

In [ ]:
pm_truth.drop('more', axis=1, inplace=True)
dataset_test=dataset_test.merge(pm_truth,on=['id'],how='left')
dataset_test['ttf']=dataset_test['rtf'] - dataset_test['cycle']
dataset_test.drop('rtf', axis=1, inplace=True)
dataset_test.head()

In [ ]:
dataset_test.shape

In [ ]:
dataset_train['ttf'] = dataset_train.groupby(['id'])['cycle'].transform(max)-dataset_train['cycle']
dataset_train.head()

In [ ]:
dataset_train.shape

In [ ]:
dataset_train['ttf'].value_counts()

In [ ]:
df_train=dataset_train.copy()
df_test=dataset_test.copy()
period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()

In [ ]:
df_train['label_bc'].value_counts()

In [ ]:
features_col_name=['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11',
                   's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
target_col_name='label_bc'

In [ ]:
sc=MinMaxScaler()
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
df_test[features_col_name]=sc.transform(df_test[features_col_name])

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train['ttf'].min()

In [ ]:
df_train['ttf'].max()

In [ ]:
df_train.iloc[0,:]

In [ ]:
import seaborn as sns
sns.pairplot(df_train)

In [ ]:
x_train = df_train.iloc[:,:-1].values
y_train = df_train.iloc[:,-1:].values

In [ ]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression()
model_log.fit(x_train,y_train)

In [ ]:
import joblib

In [ ]:
joblib.dump(model_log, "engine_model.sav")

In [ ]:
x_test = df_test.iloc[:,:-1].values
y_test = df_test.iloc[:,-1:].values

In [ ]:
y_predlog = model_log.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_predlog,y_test)

In [ ]:
df_test['label_bc'].value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y_test,y_predlog)
cm1

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib

27 inputs 1 output

unit id, cycle, 3-settings, 21 sensor data, ttf(trajectory factor)

id -> 1-100 (int) cycle -> 1-400 (int) settings, sensors -> 0-1 (float) ttf -> 0-361 (int)

In [ ]:
inp =[1,1,0.45977,0.166667,0,0,0.183735,0.406802,0.309757,0,1,0.726248,0.242424,0.109755,0,0.369048,0.633262,0.205882,0.199608,0.363986,0,0.333333,0,0,0.713178,0.724662,191]

In [ ]:
out=0

In [ ]:
model = joblib.load("engine_model.sav")

In [ ]:
model.predict([inp])

In [ ]:
import random

In [ ]:
inp1=[]
inp1.append(random.randint(0,100)) #id
inp1.append(random.randint(0,365)) #cycle
for i in range(0,24):
    inp1.append(random.uniform(0,1))
inp1.append(random.randint(0,365)) #ttf

In [ ]:
len(inp1)

In [ ]:
model.predict([inp1])

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
        "url":"https://us-south.ml.cloud.ibm.com",
        "apikey":"HkOydNLfzniWDo6QaNwNX1274Y5tJ05-xyxPq-Owbv-e" 
}
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client,'models')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.8")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model = model_log, meta_props = {
    client.repository.ModelMetaNames.NAME:"engine_model",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_0.23", 
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                             )
                                              
model_id = client.repository.get_model_uid(model_details)
                                              

In [ ]:
model_id

In [ ]:
x_train[0]

In [ ]:
model_log.predict([[1.00000000e+00, 1.00000000e+00, 4.59770115e-01, 1.66666667e-01,
       0.00000000e+00, 0.00000000e+00, 1.83734940e-01, 4.06801831e-01,
       3.09756921e-01, 0.00000000e+00, 1.00000000e+00, 7.26247987e-01,
       2.42424242e-01, 1.09755003e-01, 0.00000000e+00, 3.69047619e-01,
       6.33262260e-01, 2.05882353e-01, 1.99607803e-01, 3.63986149e-01,
       0.00000000e+00, 3.33333333e-01, 0.00000000e+00, 0.00000000e+00,
       7.13178295e-01, 7.24661696e-01, 1.91000000e+02]])